<a href="https://colab.research.google.com/github/mahireddy1318/udemy_pytorch/blob/main/05_PyTotch_going_modular_excercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1.Get Data

In [ ]:
%%writefile get_data.py
import os
import requests
from pathlib import Path
from zipfile import ZipFile

data_path = Path('data/')
image_path = data_path / 'pizza_steak_sushi'

"""Check the directory contains data, if not download from Github link

"""
if image_path.is_dir():
  print(f'{image_path} is already exists.')
else:
  print(f'{image_path} does not exists..')
  os.makedirs(image_path)
with open(data_path/'pizza_steak_sushi.zip', 'wb') as f:
  request = requests.get('https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip')
  print(f'{image_path} downloading...')
  f.write(request.content)

with ZipFile(data_path/'pizza_steak_sushi.zip', 'r') as z:
  z.extractall(image_path)

os.remove(data_path/'pizza_steak_sushi.zip')


Overwriting get_data.py


In [ ]:
!python get_data.py

data/pizza_steak_sushi is already exists.
data/pizza_steak_sushi downloading...


In [ ]:
#!rm -rf data

In [ ]:

# train_dir = Path(image_path/'train')
# test_dir = Path(image_path/'test')
# train_dir, test_dir

# 2. data_setup


In [ ]:
os.mkdir('going_modular')

FileExistsError: ignored

In [ ]:
%%writefile going_modular/data_setup.py
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

def create_dataloaders(train_dir : str,
                       test_dir : str,
                       transform : transforms.Compose,
                       batch_size : int,
                       nun_workers = os.cpu_count()
                       ):
  train_data = datasets.ImageFolder(train_dir,
                                  transform= transform)

  test_data = datasets.ImageFolder(test_dir,
                                  transform= transform)

  class_names = train_data.classes
  train_dataloader = DataLoader(train_data,
                                batch_size=batch_size,
                                shuffle= True,
                                num_workers= nun_workers,
                                pin_memory=True)

  test_dataloader = DataLoader(test_data,
                                batch_size=batch_size,
                                shuffle= True,
                                num_workers= nun_workers,
                                pin_memory=True)
  return train_dataloader, test_dataloader, class_names



Overwriting going_modular/data_setup.py


In [ ]:
# data_tranform = transforms.Compose([
#     transforms.Resize(size=(64, 64)),
#     transforms.ToTensor()
# ])
# train_dataloader, test_dataloader, class_names = create_dataloaders(train_dir=train_dir,
#                                                                     test_dir=test_dir,
#                                                                     transform=data_tranform,
#                                                                     batch_size=16
#                                                                     )
# train_dataloader, test_dataloader, class_names

In [ ]:
# img, label = next(iter(train_dataloader))
# img.shape, label.shape

# 3. Building a model

In [ ]:
# import torch
# from torch import nn

# class TinyVGG(nn.Module):
#   def __init__(self, input_shape : int, hidden_units : int, output_shape : int):
#     super().__init__()

#     self.conv1 = nn.Sequential(
#         nn.Conv2d(in_channels=input_shape, out_channels= hidden_units, kernel_size=3, stride=1, padding=0),
#         nn.ReLU(),
#         nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=3, stride=1, padding=0),
#         nn.ReLU(),
#         nn.MaxPool2d(kernel_size=2, stride=2))

#     self.conv2 = nn.Sequential(
#         nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=3, stride=1, padding=0),
#         nn.ReLU(),
#         nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=3, stride=1, padding=0),
#         nn.ReLU(),
#         nn.MaxPool2d(kernel_size=2, stride=2)
#     )

#     self.classifier = nn.Sequential(
#         nn.Flatten(),
#         nn.Linear(in_features=hidden_units, out_features= output_shape)
#     )

#   def forward(self, x : img):
#     x = self.conv1(x),
#     print(x)
#     x = self.conv2(x),
#     print(x)
#     x = self.classifier(x)
#     print(x)

#     #x = self.classifier(self.conv2(self.conv1(x)))
#     return x


In [ ]:
%%writefile going_modular/model_builder.py
import torch
from torch import nn
class TinyVGG(nn.Module):
  def __init__(self, input_shape: int, hidden_units: int, output_shape:int):
    super().__init__()

    self.conv_block1 = nn.Sequential(
        nn.Conv2d(in_channels=input_shape,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=1),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2) # default stride value is same as kernel_size
    )

    self.conv_block2 = nn.Sequential(
        nn.Conv2d(in_channels=hidden_units,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=1),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2) # default stride value is same as kernel_size
    )

    self.classifier = nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features=hidden_units*16*16,
                  out_features=output_shape)
    )

  def forward(self, x: torch.Tensor):
    x = self.conv_block1(x)
    #print(x.shape)
    x = self.conv_block2(x)
    #print(x.shape)
    x = self.classifier(x)
    #print(x.shape)
    return x
    # return self.classifier(self.conv_block_2(self.conv_block_1(x))) # <- leverage the benefits of operator fusion




Overwriting going_modular/model_builder.py


In [ ]:
# device = 'cuda' if torch.cuda.is_available() else 'cpu'
# model = TinyVGG(input_shape=3, hidden_units=10, output_shape=len(class_names)).to(device)
# model

In [ ]:
# img, label = next(iter(train_dataloader))

# model(img.to(device))

# 4. Creating train_step() and test_step() functions and train() to combine them

In [ ]:
%%writefile going_modular/engine.py
import torch
from torch import nn
from tqdm.auto import tqdm
from typing import Dict, List, Tuple

def train_step(model: torch.nn.Module,
               dataloader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               device: torch.device) -> Tuple[float, float]:
  # Put model in train mode
  model.train()

  # Setup train loss and train accuracy values
  train_loss, train_acc = 0, 0

  # Loop through data loader data batches
  for batch, (X, y) in enumerate(dataloader):
      # Send data to target device
      X, y = X.to(device), y.to(device)

      # 1. Forward pass
      y_pred = model(X)

      # 2. Calculate  and accumulate loss
      loss = loss_fn(y_pred, y)
      train_loss += loss.item()

      # 3. Optimizer zero grad
      optimizer.zero_grad()

      # 4. Loss backward
      loss.backward()

      # 5. Optimizer step
      optimizer.step()

      # Calculate and accumulate accuracy metric across all batches
      y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
      train_acc += (y_pred_class == y).sum().item()/len(y_pred)

  # Adjust metrics to get average loss and accuracy per batch
  train_loss = train_loss / len(dataloader)
  train_acc = train_acc / len(dataloader)
  return train_loss, train_acc

def test_step(model: torch.nn.Module,
              dataloader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module,
              device: torch.device) -> Tuple[float, float]:
  # Put model in eval mode
  model.eval()

  # Setup test loss and test accuracy values
  test_loss, test_acc = 0, 0

  # Turn on inference context manager
  with torch.inference_mode():
      # Loop through DataLoader batches
      for batch, (X, y) in enumerate(dataloader):
          # Send data to target device
          X, y = X.to(device), y.to(device)

          # 1. Forward pass
          test_pred_logits = model(X)

          # 2. Calculate and accumulate loss
          loss = loss_fn(test_pred_logits, y)
          test_loss += loss.item()

          # Calculate and accumulate accuracy
          test_pred_labels = test_pred_logits.argmax(dim=1)
          test_acc += ((test_pred_labels == y).sum().item()/len(test_pred_labels))

  # Adjust metrics to get average loss and accuracy per batch
  test_loss = test_loss / len(dataloader)
  test_acc = test_acc / len(dataloader)
  return test_loss, test_acc


def train(model : torch.nn.Module,
          train_dataloader : torch.utils.data.DataLoader,
          test_dataloader : torch.utils.data.DataLoader,
          loss_fn : torch.nn.Module,
          optimizer : torch.optim.Optimizer,
          epochs : int,
          device : torch.device):

  # Create empty results dictionary
  results = {"train_loss": [],
      "train_acc": [],
      "test_loss": [],
      "test_acc": []
  }
  for epoch in range(epochs):

      print('aa', epoch)
      train_loss, train_acc = train_step(model= model,dataloader=train_dataloader, loss_fn=loss_fn, optimizer=optimizer, device=device)
      test_loss, test_acc = test_step(model=model, dataloader=test_dataloader, loss_fn=loss_fn, device=device)

      print(
            f"Epoch: {epoch+1} | "
            f"train_loss: {train_loss:.4f} | "
            f"train_acc: {train_acc:.4f} | "
            f"test_loss: {test_loss:.4f} | "
            f"test_acc: {test_acc:.4f}"
        )
      results['train_loss'].append(train_loss)
      results['train_acc'].append(train_acc)
      results['test_loss'].append(test_loss)
      results['test_acc'].append(test_acc)

  return results


Overwriting going_modular/engine.py


# 5. Creating a function to save the model (utils.py)

In [ ]:
%%writefile going_modular/utils.py
import torch
from pathlib import Path
def save_model(
    model : torch.nn.Module,
    target_dir : str,
    model_name : str):

  target_dir_path = Path(target_dir)
  target_dir_path.mkdir(parents=True, exist_ok=True)

  # Create model save path
  assert model_name.endswith(".pth") or model_name.endswith(".pt"), "model_name should end with '.pt' or '.pth'"
  model_save_path = target_dir_path / model_name

  print(f"[INFO] Saving model to: {model_save_path}")
  torch.save(obj=model.state_dict(),
             f=model_save_path)


Overwriting going_modular/utils.py


# 6. Train, evaluate and save the model (train.py)

In [ ]:
%%writefile going_modular/train.py
"""
Trains a PyTorch image classification model using device-agnostic code.
"""

import os
import torch
import data_setup, engine, model_builder, utils

from torchvision import transforms

# Setup hyperparameters
NUM_EPOCHS = 5
BATCH_SIZE = 32
HIDDEN_UNITS = 10
LEARNING_RATE = 0.001

# Setup directories
train_dir = "data/pizza_steak_sushi/train"
test_dir = "data/pizza_steak_sushi/test"

# Setup target device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Create transforms
data_transform = transforms.Compose([
  transforms.Resize((64, 64)),
  transforms.ToTensor()
])

# Create DataLoaders with help from data_setup.py
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
    train_dir=train_dir,
    test_dir=test_dir,
    transform=data_transform,
    batch_size=BATCH_SIZE
)

# Create model with help from model_builder.py
model = model_builder.TinyVGG(
    input_shape=3,
    hidden_units=HIDDEN_UNITS,
    output_shape=len(class_names)
).to(device)

# Set loss and optimizer
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),
                             lr=LEARNING_RATE)

# Start training with help from engine.py
engine.train(model=model,
             train_dataloader=train_dataloader,
             test_dataloader=test_dataloader,
             loss_fn=loss_fn,
             optimizer=optimizer,
             epochs=NUM_EPOCHS,
             device=device)

# Save the model with help from utils.py
utils.save_model(model=model,
                 target_dir="models",
                 model_name="05_going_modular_script_mode_tinyvgg_model.pth")


Overwriting going_modular/train.py


In [ ]:
%%writefile going_modular/train_new_parse.py
import os
import argparse

import torch

from torchvision import transforms
import data_setup, engine, model_builder, utils

# Create a parser
parser = argparse.ArgumentParser(description="Get some hyperparameters.")

# Get an arg for num_epochs
parser.add_argument("--num_epochs",
                     default=10,
                     type=int,
                     help="the number of epochs to train for")

# Get an arg for batch_size
parser.add_argument("--batch_size",
                    default=32,
                    type=int,
                    help="number of samples per batch")

# Get an arg for hidden_units
parser.add_argument("--hidden_units",
                    default=10,
                    type=int,
                    help="number of hidden units in hidden layers")

# Get an arg for learning_rate
parser.add_argument("--learning_rate",
                    default=0.001,
                    type=float,
                    help="learning rate to use for model")

# Create an arg for training directory
parser.add_argument("--train_dir",
                    default="data/pizza_steak_sushi/train",
                    type=str,
                    help="directory file path to training data in standard image classification format")

# Create an arg for test directory
parser.add_argument("--test_dir",
                    default="data/pizza_steak_sushi/test",
                    type=str,
                    help="directory file path to testing data in standard image classification format")

# Get our arguments from the parser
args = parser.parse_args()

# Setup hyperparameters
NUM_EPOCHS = args.num_epochs
BATCH_SIZE = args.batch_size
HIDDEN_UNITS = args.hidden_units
LEARNING_RATE = args.learning_rate
print(f"[INFO] Training a model for {NUM_EPOCHS} epochs with batch size {BATCH_SIZE} using {HIDDEN_UNITS} hidden units and a learning rate of {LEARNING_RATE}")

# Setup directories
train_dir = args.train_dir
test_dir = args.test_dir
print(f"[INFO] Training data file: {train_dir}")
print(f"[INFO] Testing data file: {test_dir}")

# Setup target device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Create transforms
data_transform = transforms.Compose([
  transforms.Resize((64, 64)),
  transforms.ToTensor()
])

# Create DataLoaders with help from data_setup.py
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
    train_dir=train_dir,
    test_dir=test_dir,
    transform=data_transform,
    batch_size=BATCH_SIZE
)

# Create model with help from model_builder.py
model = model_builder.TinyVGG(
    input_shape=3,
    hidden_units=HIDDEN_UNITS,
    output_shape=len(class_names)
).to(device)

# Set loss and optimizer
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),
                             lr=LEARNING_RATE)

# Start training with help from engine.py
engine.train(model=model,
             train_dataloader=train_dataloader,
             test_dataloader=test_dataloader,
             loss_fn=loss_fn,
             optimizer=optimizer,
             epochs=NUM_EPOCHS,
             device=device)

# Save the model with help from utils.py
utils.save_model(model=model,
                 target_dir="models",
                 model_name="05_going_modular_script_mode_tinyvgg_model.pth")


Overwriting going_modular/train_new_parse.py


In [ ]:
!python /content/going_modular/train_new_parse.py --num_epochs 5 --batch_size 16 --hidden_units 128 --learning_rate 0.0003 --train_dir '/content/data/pizza_steak_sushi/train' --test_dir '/content/data/pizza_steak_sushi/test'

[INFO] Training a model for 5 epochs with batch size 16 using 128 hidden units and a learning rate of 0.0003
[INFO] Training data file: /content/data/pizza_steak_sushi/train
[INFO] Testing data file: /content/data/pizza_steak_sushi/test
aa 0
Epoch: 1 | train_loss: 1.0975 | train_acc: 0.3833 | test_loss: 1.0948 | test_acc: 0.3523
aa 1
Epoch: 2 | train_loss: 1.0688 | train_acc: 0.4125 | test_loss: 1.0426 | test_acc: 0.4295
aa 2
Epoch: 3 | train_loss: 0.9220 | train_acc: 0.5875 | test_loss: 1.2709 | test_acc: 0.4341
aa 3
Epoch: 4 | train_loss: 0.9596 | train_acc: 0.5625 | test_loss: 1.0017 | test_acc: 0.4852
aa 4
Epoch: 5 | train_loss: 0.8675 | train_acc: 0.5500 | test_loss: 0.9537 | test_acc: 0.4477
[INFO] Saving model to: models/05_going_modular_script_mode_tinyvgg_model.pth


In [ ]:
!rm -rf models